## Run DO.ipynb until Checkpoint#1

In [ ]:
import syft as sy
import torch
from syft.core.remote_dataloader import RemoteDataset
from syft.core.remote_dataloader import RemoteDataLoader

In [ ]:
duet = sy.join_duet(loopback=True)

## Checkpoint#1

## Run DO.ipynb until the end

In [ ]:
meta_ptr = duet.store["meta"]

In [ ]:
duet.store.pandas

In [ ]:
# create RemoteDataset object on remote side
rds_ptr = duet.syft.core.remote_dataloader.RemoteDataset(meta_ptr)
# create RemoteDataLoader object on remote side

In [ ]:
rdl_ptr = duet.syft.core.remote_dataloader.RemoteDataLoader(rds_ptr, batch_size=32)
# call create_dataset to create the real Dataset object on remote side
rdl_ptr.load_dataset()
# call create_dataloader to create the real DataLoader object on remote side
rdl_ptr.create_dataloader()

In [ ]:
class Model(sy.Module):
    def __init__(self, torch_ref):
        super(Model, self).__init__(torch_ref=torch_ref)   
        
        self.layer = self.torch_ref.nn.Linear(13, 1)

    def forward(self, x):
        output = self.layer(x)
        return output

local_model = Model(torch)
remote_model = local_model.send(duet)

In [ ]:
# you can use rdl_ptr very similar to a normal DataLoader object
for epoch in range(2):
    epoch_loss = 0
    for i,b in enumerate(rdl_ptr):
        optimizer = duet.torch.optim.Adam(params=remote_model.parameters(),lr=1e-2)
        optimizer.zero_grad()    
        X, y = b[0], b[1]
        pred = remote_model(X)    
        loss = ((pred.view(-1) - y)**2).mean()
        loss.backward()
        optimizer.step()
        gotloss = loss.get(request_block=True,delete_obj= False, timeout_secs=2)
        print("[epoch#{}-iter#{:2}] : loss={:.2f}".format(epoch, i, gotloss))